# Podcast with Map Reduce

This example introduces the Map Reduce pattern and the text-to-speech task.
The map reduce pattern is useful to run parallel activities which are then combined into a final result.
In this case, we use it to generate audio for each phrase in the transcript and then combine them into a final podcast.

In [1]:
from audio_processing import split_transcript, combine_audio_files

from content_composer.registry import RegistryScope, get_registry

registry = get_registry()
    
# Register the function in the registry
registry.register(
    identifier="split_transcript",
    function=split_transcript,
    description="Splits a transcript into phrases with speaker and voice information",
    tags=["transcript", "splitting", "audio"],
)

registry.register(
    identifier="combine_audio_files",
    function=combine_audio_files,
    description="Combines individual audio files into a single podcast audio file",
    tags=["audio", "combining", "podcast"]
)
    


2025-06-20 17:48:43.095 | INFO     | content_composer.registry.registry:initialize:26 - Function registry initialized with 0 functions


True

In [2]:
from content_composer import parse_recipe, execute_workflow

# workflows can be defined in multiple formats, this example uses a json file
recipe = parse_recipe("./full_podcast.yaml")
recipe

Recipe(name='Podcast Generation Test', user_inputs=[UserInput(id='briefing', label='Briefing', type='text', description='The briefing for the podcast', default="Let's talk about ...", required=True, literal_values=None), UserInput(id='output_filename', label='Output Filename for Podcast', type='string', description='Desired filename for the final combined podcast audio (e.g., podcast_episode.mp3).', default='podcast_test_output.mp3', required=True, literal_values=None), UserInput(id='default_voice_id', label='Voice ID for Speaker A / Fallback Default', type='string', description='Voice ID for Speaker A, and fallback for any unmapped speakers.', default='wbZYPUM3zIb4exd1P7a0', required=True, literal_values=None), UserInput(id='speaker_b_voice_id', label='Voice ID for Speaker B', type='string', description='Enter the voice ID to use for Speaker B.', default='CwhRBWXzGAHq8TQ4Fs17', required=True, literal_values=None)], nodes=[Node(id='generate_transcript', type=<NodeType.LANGUAGE_TASK: 'l

In [3]:
inputs = {"briefing": "Joana is an experienced journalist and she is interviewing Marc, who believes we are leaving in a simulation", 
    "default_voice_id": "wbZYPUM3zIb4exd1P7a0", 
    "speaker_b_voice_id": "CwhRBWXzGAHq8TQ4Fs17"}

outputs = await execute_workflow(
    recipe,
    inputs
)


2025-06-20 17:48:55.859 | INFO     | audio_processing:split_transcript:25 - [Core Function] split_transcript called. Voice Mapping: {'Speaker A': 'wbZYPUM3zIb4exd1P7a0', 'Speaker B': 'CwhRBWXzGAHq8TQ4Fs17'}, Default Voice ID: wbZYPUM3zIb4exd1P7a0
2025-06-20 17:48:55.861 | INFO     | audio_processing:split_transcript:79 - split_transcript: Successfully split transcript into 19 phrases.
2025-06-20 17:49:48.421 | INFO     | audio_processing:combine_audio_files:94 - [Core Function] combine_audio_files called.


MoviePy - Writing audio in output/audio/podcast_test_output.mp3


2025-06-20 17:49:51.221 | INFO     | audio_processing:combine_audio_files:159 - Successfully combined audio to: /Users/luisnovo/dev/projetos/content-composer/examples/04_map_reduce_podcast/output/audio/podcast_test_output.mp3


MoviePy - Done.


In [4]:
outputs

{'briefing': 'Joana is an experienced journalist and she is interviewing Marc, who believes we are leaving in a simulation',
 'default_voice_id': 'wbZYPUM3zIb4exd1P7a0',
 'speaker_b_voice_id': 'CwhRBWXzGAHq8TQ4Fs17',
 'output_filename': 'podcast_test_output.mp3',
 'full_transcript': '```\nSpeaker A: Welcome to today’s episode of "Beyond Reality." I’m thrilled to have Marc with us, someone who has a very unique perspective on the nature of our existence. Marc, thanks for joining us.\nSpeaker B: Thanks for having me. I’m excited to dive into this topic.\nSpeaker A: So, Marc, let’s start with the big idea. You believe we’re living in a simulation. Can you explain what that means to you?\nSpeaker B: Absolutely. The simulation hypothesis suggests that what we perceive as reality is actually a highly advanced computer simulation, kind of like a super-sophisticated video game. Everything we experience—our world, our emotions, even our thoughts—could be coded by a higher intelligence or civili

In [10]:
outputs["final_podcast_details"]["combined_audio_path"].split("/")[-1]

'podcast_test_output.mp3'